# Test GPU Training
I was getting the same error here: https://www.reddit.com/r/tensorflow/comments/dxnnq2/i_am_getting_an_error_while_running_the_rnn_lstm/f7wv17k/, and used the example code from the thread to debug.

### Import Tensorflow and Check GPU Support

In [1]:
import tensorflow as tf
print('Built with Cuda: {}'.format(tf.test.is_built_with_cuda()))
print('Build with GPU support: {}'.format(tf.test.is_built_with_gpu_support()))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Built with Cuda: True
Build with GPU support: True
Num GPUs Available:  1


### Build Model

In [2]:
def build_model(allow_cudnn_kernel=True):
    if allow_cudnn_kernel:
        lstm_layer = tf.keras.layers.LSTM(units, input_shape=(None, input_dim))
    else:
        lstm_layer = tf.keras.layers.RNN(tf.keras.layers.LSTMCell(units),
                                         input_shape=(None, input_dim))
    model = tf.keras.models.Sequential([
        lstm_layer,
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(output_size, activation='softmax')]
    )
    return model

### Load Data and Train MNIST

In [3]:
batch_size = 64
input_dim = 28
units = 64
output_size = 10

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

sample, sample_label = x_train[0], y_train[0]
model = build_model(allow_cudnn_kernel=True)

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='sgd',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=batch_size,
          epochs=5)

Train on 60000 samples, validate on 10000 samples
Epoch 1/5
60000/60000 [==============================] - 13s 210us/sample - loss: 0.8810 - accuracy: 0.7252 - val_loss: 0.5482 - val_accuracy: 0.8137
Epoch 2/5
60000/60000 [==============================] - 8s 141us/sample - loss: 0.3803 - accuracy: 0.8862 - val_loss: 0.5401 - val_accuracy: 0.8105
Epoch 3/5
60000/60000 [==============================] - 8s 141us/sample - loss: 0.2545 - accuracy: 0.9237 - val_loss: 0.2623 - val_accuracy: 0.9185
Epoch 4/5
60000/60000 [==============================] - 9s 155us/sample - loss: 0.2001 - accuracy: 0.9401 - val_loss: 0.7232 - val_accuracy: 0.7656
Epoch 5/5
60000/60000 [==============================] - 9s 155us/sample - loss: 0.1701 - accuracy: 0.9481 - val_loss: 0.1583 - val_accuracy: 0.9486
